In [5]:
# System
import os
import sys
sys.path.append('/home/helfrech/Tools/Toolbox/utils')

# Maths
import numpy as np

# ML
from regression import SparseKRR
from kernels import build_kernel, linear_kernel, gaussian_kernel

# Utilities
import h5py

In [1]:
# TODO: load cutoffs, gaussian widths, regularizations, and sigmas

In [ ]:
# TODO: load train/test indices

In [ ]:
# TODO: load representative indices and build representative environments for each cutoff

In [ ]:
# TODO: put reusable code into functions in within this notebook

In [ ]:
# TODO: pickle the deem regression models

In [ ]:
# TODO: functions for DEEM/IZA/COD and IZAonDEEM/CODonDEEM

In [ ]:
property_names = ['volumes', 'energies']

# DEEM_10k

In [ ]:
deem_10k_structure_volumes = np.loadtxt('../Processed_Data/DEEM_10k/structure_volumes.dat')
deem_10k_structure_energies = np.loadtxt('../Processed_Data/DEEM_10k/structure_energies.dat')
deem_10k_structure_properties = [deem_10k_structure_volumes, deem_10k_structure_energies]

In [ ]:
for cutoff in cutoffs:
    
    deem_10k = []
        
    # Read SOAPs from HDF5
    f = h5py.File('../Processed_Data/DEEM_10k/KPCA/{cutoff}/soaps.hdf5', 'r')
    for soap in f.values():
        deem_10k.append(soap)
    f.close()

    # Build kernels
    KMM = build_kernel(representative_soaps, representative_soaps,
                       kernel=kernel_type, **kernel_parameters)
    KNM_train = build_kernel([deem_10k[i] for i in train_idxs], representative_soaps,
                             kernel=kernel_type, **kernel_parameters)
    KNM_test = build_kernel([deem_10k[i] for i in test_idxs], representative_soaps,
                            kernel=kernel_type, **kernel_parameters)
    KNM_environments = build_kernel(np.concatenate(deem_10k, axis=0), representative_soaps,
                                  kernel=kernel_type, **kernel_parameters)
    
    for pn, Y in zip(property_names, deem_10k_structure_properties):
        
        Yp_structures = np.zeros(len(deem_10k))
        delta = np.var(Y[train_idxs]) * KMM.shape[0] / np.trace(KMM)
    
        # Initialize sparse KRR
        skrr = SparseKRR(sigma=sigma, reg=reg, rcond=rcond)
        skrr.fit(delta*KNM_train, delta*KMM, delta*Y[train_idxs])
        
        Yp_structures[train_idxs] = skrr.transform(KNM_train)
        Yp_structures[test_idxs] = skrr.transform(KNM_test)
        Yp_environments = skrr.transform(KNM_environments)
        
        np.savetxt('../Processed_Data/DEEM_10k/KPCA/{cutoff}/predicted_structure_{pn}.dat', Yp_structures)
        np.savetxt('../Processed_Data/DEEM_10k/KPCA/{cutoff}/predicted_environment_{pn}.dat', Yp_environments)

# IZA_226

In [ ]:
iza_226_structure_volumes = np.loadtxt('../Processed_Data/IZA_226/structure_volumes.dat')
iza_226_structure_energies = np.loadtxt('../Processed_Data/IZA_226/structure_energies.dat')
iza_226_structure_properties = [iza_226_structure_volumes, iza_226_structure_energies]

## Pure IZA_226

## IZA_226 on DEEM_10k

In [ ]:
for cutoff in cutoffs:
        
    iza_226 = []
    
    # Read SOAPs from HDF5
    f = h5py.File('../Processed_Data/IZA_226onDEEM_10k/KPCA/{cutoff}/soaps.hdf5', 'r')
    for soap in f.values():
        iza_226.append(soap)
    f.close()
    
    for pn in property_names:
        KNM_iza = build_kernel(iza_226, representative_soaps,
                               kernel=kernel_type, **kernel_parameters)
        K_environments = build_kernel(np.concatenate(iza_226, axis=0), representative_soaps,
                                      kernel=kernel_type, **kernel_parameters)
    
        # Predict based on DEEM_10k
        # TODO: make sure to grab the correct iskrr object
        Yp_structures = iskrr.transform(KNM_iza)
        Yp_environments = iskrr.transform(K_environments)
        
        np.savetxt('../Processed_Data/IZA_226onDEEM_10k/KPCA/{cutoff}/predicted_structure_{pn}.dat', Yp_structures)
        np.savetxt('../Processed_Data/IZA_226onDEEM_10k/KPCA/{cutoff}/predicted_environment_{pn}.dat', Yp_environments)

# COD_196

In [ ]:
cod_196_structure_volumes = np.loadtxt('../Processed_Data/COD_196/structure_volumes.dat')
cod_196_strcuture_energies = np.loadtxt('../Processed_Data/COD_196/structure_energies.dat')
cod_196_structure_properties = [cod_196_structure_volumes, cod_196_structure_energies]

## Pure COD_196

## COD_196 on DEEM_10k